In [1]:
import torch
from torch import nn 
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.4137,  0.2021,  0.0584,  0.2482,  0.0942,  0.1700,  0.0306,  0.2328,
          0.0599,  0.1214],
        [ 0.2796, -0.0109, -0.0038,  0.2739,  0.1104,  0.1726, -0.0668,  0.2318,
         -0.0871,  0.0676]], grad_fn=<AddmmBackward>)

In [3]:
net[2].weight.shape

torch.Size([10, 256])

### 自定义块

In [4]:
class MLP(nn.Module):
    
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        # 必须重写forward，实现各个层之间的连接关系，
        # 只要定义了forward函数，backward函数会被自动实现
        return self.out(F.relu(self.hidden(X)))
    

In [5]:
net = MLP()
net(X)

tensor([[ 0.2291,  0.1552,  0.1471,  0.0059, -0.3268,  0.0794, -0.0952, -0.4270,
          0.1956, -0.2151],
        [-0.0112,  0.1319,  0.0930,  0.0300, -0.2132,  0.0519, -0.0343, -0.3415,
          0.1546, -0.1686]], grad_fn=<AddmmBackward>)

### 顺序块

In [6]:
class MySquential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # _modules 里面存着 Module 的实例，为一个OrderDict
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)       
        return X

In [7]:
net = MySquential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2693, -0.2171, -0.0084, -0.0377, -0.1736,  0.0932,  0.0772, -0.1582,
         -0.0167, -0.0280],
        [-0.1433, -0.0832,  0.0818, -0.0546, -0.0189,  0.1212,  0.1298, -0.1778,
          0.1196,  0.0428]], grad_fn=<AddmmBackward>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随即权重参数， 在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1550, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

In [11]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2038, grad_fn=<SumBackward0>)

In [12]:
# test
class BigBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.net1 = nn.Linear(20,60)
        self.net2 = nn.Linear(20,40)
    
    def forward(self, X):
        X1 = self.net1(X)
        X2 = self.net2(X)
        X3 = torch.cat((X1, X2), 1)
        net = nn.Linear(X3.shape[1], 22)
        return net(X3)

In [13]:
net = BigBlock()
net(X).shape

torch.Size([2, 22])